# Download artifacts from wandb

In [ ]:
!pip install wandb huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.7/214.7 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=a4020a6d81487cfc735e82f30550b3988cf86855c11afc61e739c6880d56191c
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
run = wandb.init()
artifact = run.use_artifact('lemontea-tom/qlora-test/checkpoint-l76a05uz:v6', type='model') #400
artifact_dir = artifact.download()

wandb: Currently logged in as: lemontea-tom. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact checkpoint-l76a05uz:v6, 145.89MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:2.3


In [ ]:
#artifact_dir = './artifacts/checkpoint-l76a05uz:v6'

In [ ]:
!ls ./artifacts/checkpoint-l76a05uz:v6

adapter_config.json  optimizer.pt  rng_state.pth  trainer_state.json
adapter_model.bin    README.md	   scheduler.pt   training_args.bin


# Merge Qlora with base model

In [ ]:
!pip install torch peft transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.6 MB/s eta 0:00:00


In [ ]:
#base_model = "openlm-research/open_llama_3b"

#from huggingface_hub import snapshot_download
#path = snapshot_download(base_model) #local_dir=IMAGE_MODEL_DIR

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, LlamaForCausalLM

config = PeftConfig.from_pretrained(artifact_dir)
base_model = LlamaForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    torch_dtype='auto',
    device_map='cpu',
    #use_safetensors=True
    #offload_folder="offload", offload_state_dict = True
)
#tokenizer = LlamaTokenizer.from_pretrained(config.base_model_name_or_path)

In [ ]:
#config = PeftConfig.from_pretrained(artifact_dir)

In [ ]:
config

PeftConfig(peft_type='LORA', auto_mapping=None, base_model_name_or_path='openlm-research/open_llama_3b', revision=None, task_type='CAUSAL_LM', inference_mode=True)

In [ ]:
#model = PeftModel.from_pretrained(base_model, artifact_dir, offload_folder="offload", offload_state_dict = True)
model = PeftModel.from_pretrained(base_model, artifact_dir)

In [ ]:
merged_model = model.merge_and_unload()

In [ ]:
merged_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 3200, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3200, out_features=3200, bias=False)
          (k_proj): Linear(in_features=3200, out_features=3200, bias=False)
          (v_proj): Linear(in_features=3200, out_features=3200, bias=False)
          (o_proj): Linear(in_features=3200, out_features=3200, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3200, out_features=8640, bias=False)
          (up_proj): Linear(in_features=3200, out_features=8640, bias=False)
          (down_proj): Linear(in_features=8640, out_features=3200, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm(

In [ ]:
merged_model.save_pretrained("merged_model2",
                             #safe_serialization=True,
                             push_to_hub=True,
                             repo_id="lemonteaa/exercise-openllama-3b-qlora-axolotl-checkpoint400-merged")

pytorch_model.bin:   0%|          | 0.00/6.85G [00:00<?, ?B/s]

In [ ]:
!ls merged_model2

config.json  generation_config.json  pytorch_model.bin


# Push model to HF

In [24]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
merged_model.push_to_hub("lemonteaa/exercise-openllama-3b-qlora-axolotl-checkpoint400-merged")

pytorch_model.bin:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/lemonteaa/exercise-openllama-3b-qlora-axolotl-checkpoint400-merged/commit/3ed1ec1e0688e445dca47035052312d09ccf6349', commit_message='Upload LlamaForCausalLM', commit_description='', oid='3ed1ec1e0688e445dca47035052312d09ccf6349', pr_url=None, pr_revision=None, pr_num=None)

Need to restart kernel at this point

In [ ]:
#del merged_model, model, base_model, config

In [ ]:
#import gc
#gc.collect()

1000

# Test loading model from HF?

In [ ]:
!ls offload

base_model.model.lm_head.weight.dat
base_model.model.model.layers.14.input_layernorm.weight.dat
base_model.model.model.layers.14.mlp.down_proj.lora_A.default.weight.dat
base_model.model.model.layers.14.mlp.down_proj.lora_B.default.weight.dat
base_model.model.model.layers.14.mlp.down_proj.weight.dat
base_model.model.model.layers.14.mlp.gate_proj.lora_A.default.weight.dat
base_model.model.model.layers.14.mlp.gate_proj.lora_B.default.weight.dat
base_model.model.model.layers.14.mlp.gate_proj.weight.dat
base_model.model.model.layers.14.mlp.up_proj.lora_A.default.weight.dat
base_model.model.model.layers.14.mlp.up_proj.lora_B.default.weight.dat
base_model.model.model.layers.14.mlp.up_proj.weight.dat
base_model.model.model.layers.14.post_attention_layernorm.weight.dat
base_model.model.model.layers.14.self_attn.k_proj.lora_A.default.weight.dat
base_model.model.model.layers.14.self_attn.k_proj.lora_B.default.weight.dat
base_model.model.model.layers.14.self_attn.k_proj.weight.dat
base_model.model

In [ ]:
!pip install sentencepiece bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.4 MB/s eta 0:00:00


In [ ]:
import torch

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openlm-research/open_llama_3b", use_fast=False)
model = AutoModelForCausalLM.from_pretrained("lemonteaa/exercise-openllama-3b-qlora-axolotl-checkpoint400-merged",
                                             #torch_dtype=torch.float16,
                                             torch_dtype='auto',
                                             device_map="auto",
                                             offload_folder="offload2", offload_state_dict = True)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
prompt = """Below is a conversation between user and assistent. The assistent is helpful and skillful.
User: Hi! How are you today?
Assistent: I'm feeling good! Anything I may help you with?
User: Write a short essay that analyze ritual from a cultural and anthropology framework.
Assistent: Sure! Let's start with the definition of ritual. According to anthropologists, ritual is a set of actions that are performed in a prescribed manner, usually with the intention of achieving a desired outcome. Rituals are often associated with religious or spiritual practices, but they can also be used in non-religious contexts, such as weddings, funerals, or celebrations.
In a cultural context, rituals are often seen as a way for people to connect with their traditions, beliefs, and values. They can help to reinforce social norms, transmit knowledge, and transmit values. Rituals can also serve as a way for people to express their identity and their place in the community.
Anthropologists believe that rituals are a way for people to make sense of the world around them. They can help to make sense of the past, present, and future, and to make sense of the relationships between people and the natural world. Rituals can also help to create a sense of community and belonging, and to reinforce social norms and values.
In conclusion, rituals are a powerful tool for people to connect with their traditions, beliefs, and values, and to make sense of the world around them. They can help to create a sense of community and belonging, and to reinforce social norms and values.
User: Thanks. I heard that primitive tribes throughout the world have diversity in their rituals, and that some tribes are harsher than others in terms of the rite of passage for their members. Postulate some possible factors explaining this variance.
Assistent: """
response = pipe(prompt, max_new_tokens = 512)
print(response[0]['generated_text'])

Below is a conversation between user and assistent. The assistent is helpful and skillful.
User: Hi! How are you today?
Assistent: I'm feeling good! Anything I may help you with?
User: Write a short essay that analyze ritual from a cultural and anthropology framework.
Assistent: Sure! Let's start with the definition of ritual. According to anthropologists, ritual is a set of actions that are performed in a prescribed manner, usually with the intention of achieving a desired outcome. Rituals are often associated with religious or spiritual practices, but they can also be used in non-religious contexts, such as weddings, funerals, or celebrations.
In a cultural context, rituals are often seen as a way for people to connect with their traditions, beliefs, and values. They can help to reinforce social norms, transmit knowledge, and transmit values. Rituals can also serve as a way for people to express their identity and their place in the community.
Anthropologists believe that rituals are

# GPTQ quantization

In [ ]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [ ]:
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
#BUILD_CUDA_EXT=0

In [ ]:
!BUILD_CUDA_EXT=1 pip install --upgrade --force-reinstall auto-gptq

  Using cached auto_gptq-0.3.2.tar.gz (63 kB)
  Preparing metadata (setup.py) ... done
Discarding https://files.pythonhosted.org/packages/1b/79/5a3a7d877a9b0a72f528e9977ec65cdb9fad800fa4f5110f87f2acaaf6fe/auto_gptq-0.3.2.tar.gz (from https://pypi.org/simple/auto-gptq/) (requires-python:>=3.8.0): Requested auto-gptq from https://files.pythonhosted.org/packages/1b/79/5a3a7d877a9b0a72f528e9977ec65cdb9fad800fa4f5110f87f2acaaf6fe/auto_gptq-0.3.2.tar.gz has inconsistent version: expected '0.3.2', but metadata has '0.3.2+cu118'
  Using cached auto_gptq-0.3.1.tar.gz (63 kB)
  Preparing metadata (setup.py) ... done
Discarding https://files.pythonhosted.org/packages/3f/5c/28d57f833498e014e9d066ea0199a503b5e59d4c8e8e701a460223e143da/auto_gptq-0.3.1.tar.gz (from https://pypi.org/simple/auto-gptq/) (requires-python:>=3.8.0): Requested auto-gptq from https://files.pythonhosted.org/packages/3f/5c/28d57f833498e014e9d066ea0199a503b5e59d4c8e8e701a460223e143da/auto_gptq-0.3.1.tar.gz has inconsistent vers

In [ ]:
!pip uninstall -y auto-gptq

Found existing installation: auto-gptq 0.3.0
Uninstalling auto-gptq-0.3.0:
  Successfully uninstalled auto-gptq-0.3.0


In [ ]:
!git clone https://github.com/PanQiWei/AutoGPTQ.git

Cloning into 'AutoGPTQ'...
remote: Enumerating objects: 2487, done.
remote: Counting objects: 100% (865/865), done.
remote: Compressing objects: 100% (423/423), done.
remote: Total 2487 (delta 543), reused 619 (delta 426), pack-reused 1622
Receiving objects: 100% (2487/2487), 7.47 MiB | 5.39 MiB/s, done.
Resolving deltas: 100% (1630/1630), done.


In [ ]:
!ls

sample_data


In [ ]:
!cd AutoGPTQ && pip install .

Processing /content/AutoGPTQ
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.6 MB/s eta 0:00:00
  Created wheel for auto-gptq: filename=auto_gptq-0.3.2+cu118-cp310-cp310-linux_x86_64.whl size=5667846 sha256=ee2559dd0570658

In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.0 MB/s eta 0:00:00


In [ ]:
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:


tokenizer = AutoTokenizer.from_pretrained("openlm-research/open_llama_3b", use_fast=False)

test1 = """Below is a conversation between user and assistent. The assistent is helpful and skillful.
User: Hi! How are you today?
Assistent: I'm feeling good! Anything I may help you with?
User: Write a short essay that analyze ritual from a cultural and anthropology framework.
Assistent:  Sure! Let's start with the definition of ritual. According to anthropologists, ritual is a set of actions that are performed in a prescribed manner, usually with the intention of achieving a desired outcome. Rituals are often associated with religious or spiritual practices, but they can also be used in non-religious contexts, such as weddings, funerals, or celebrations.
In a cultural context, rituals are often seen as a way for people to connect with their traditions, beliefs, and values. They can help to reinforce social norms, transmit knowledge, and transmit values. Rituals can also serve as a way for people to express their identity and their place in the community.
Anthropologists believe that rituals are a way for people to make sense of the world around them. They can help to make sense of the past, present, and future, and to make sense of the relationships between people and the natural world. Rituals can also help to create a sense of community and belonging, and to reinforce social norms and values.
In conclusion, rituals are a powerful tool for people to connect with their traditions, beliefs, and values, and to make sense of the world around them. They can help to create a sense of community and belonging, and to reinforce social norms and values.
User: Thanks. I heard that primitive tribes throughout the world have diversity in their rituals, and that some tribes are harsher than others in terms of the rite of passage for their members. Postulate some possible factors explaining this variance.
Assistent:  Yes, primitive tribes throughout the world have a wide range of rituals, and some tribes are harsher than others in terms of the rite of passage for their members. Some of the possible factors that may explain this variance include the availability of resources, the level of social organization, and the level of technology.
In societies where resources are scarce, it may be more important for people to have a clear and defined set of rituals that help to reinforce social norms and values. In societies with a high level of social organization, it may be more important for people to have a clear and defined set of rituals that help to reinforce social norms and values. In societies with a high level of technology, it may be more important for people to have a clear and defined set of rituals that help to reinforce social norms and values.
In conclusion, the factors that may explain the variance in the rite of passage for members of primitive tribes throughout the world include the availability of resources, the level of social organization, and the level of technology."""

examples = [
    tokenizer(test1)
]

quantize_config = BaseQuantizeConfig(
    bits=4,
    group_size=128,
    desc_act=False,
)

#model = AutoGPTQForCausalLM.from_pretrained("lemonteaa/exercise-openllama-3b-qlora-axolotl-checkpoint400-merged", quantize_config)
#model.quantize(examples)

#model.push_to_hub("lemonteaa/exercise-openllama-3b-qlora-axolotl-checkpoint400-GPTQ", save_dir="gptq-model1", use_safetensors=True)

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [ ]:
model = AutoGPTQForCausalLM.from_pretrained("lemonteaa/exercise-openllama-3b-qlora-axolotl-checkpoint400-merged",
                                            quantize_config,
                                            max_memory={0:'14GiB', 'cpu': '10GiB'})

In [ ]:
%cd AutoGPTQ

/content/AutoGPTQ


In [ ]:
import autogptq_cuda

In [ ]:
model.quantize(examples)

In [ ]:
model.push_to_hub("lemonteaa/exercise-openllama-3b-qlora-axolotl-checkpoint400-GPTQ",
                  save_dir="gptq-model1", use_safetensors=True)

gptq_model-4bit-128g.safetensors:   0%|          | 0.00/2.09G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/lemonteaa/exercise-openllama-3b-qlora-axolotl-checkpoint400-GPTQ/commit/129dd3e26797dd678a7fc76ebd30b1eb997448f9', commit_message='Upload of AutoGPTQ quantized model', commit_description='', oid='129dd3e26797dd678a7fc76ebd30b1eb997448f9', pr_url=None, pr_revision=None, pr_num=None)

# GGML Quantization

In [1]:
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 3.4 MB/s eta 0:00:00


In [2]:
!mkdir my_model

In [3]:
from huggingface_hub import snapshot_download

my_model = "lemonteaa/exercise-openllama-3b-qlora-axolotl-checkpoint400-merged"

path = snapshot_download(my_model, local_dir="/content/my_model/") #local_dir=IMAGE_MODEL_DIR

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

In [13]:
from huggingface_hub import hf_hub_download
tok_path = hf_hub_download(repo_id="openlm-research/open_llama_3b", filename="tokenizer.model", local_dir="/content/my_model/")

In [15]:
tok_path

'/content/mymodel/tokenizer.model'

In [4]:
!git clone https://github.com/ggerganov/llama.cpp

Cloning into 'llama.cpp'...
remote: Enumerating objects: 5692, done.
remote: Counting objects: 100% (2350/2350), done.
remote: Compressing objects: 100% (332/332), done.
remote: Total 5692 (delta 2228), reused 2043 (delta 2016), pack-reused 3342
Receiving objects: 100% (5692/5692), 4.42 MiB | 7.01 MiB/s, done.
Resolving deltas: 100% (3931/3931), done.


In [4]:
%cd llama.cpp

/content/llama.cpp


In [6]:
!make

I llama.cpp build info: 
I UNAME_S:  Linux
I UNAME_P:  x86_64
I UNAME_M:  x86_64
I CFLAGS:   -I.              -O3 -std=c11   -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wdouble-promotion -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -pthread -march=native -mtune=native -DGGML_USE_K_QUANTS
I CXXFLAGS: -I. -I./examples -O3 -std=c++11 -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wno-multichar -pthread -march=native -mtune=native -DGGML_USE_K_QUANTS
I LDFLAGS:  
I CC:       cc (Ubuntu 11.3.0-1ubuntu1~22.04.1) 11.3.0
I CXX:      g++ (Ubuntu 11.3.0-1ubuntu1~22.04.1) 11.3.0

cc  -I.              -O3 -std=c11   -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wdouble-promotion -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -pthread -march=native -mtune=native -DGGML_USE_K_QUANTS   -c ggml.c -o ggml.o
g++ -I. -I./examples -O3 -std=c++11 -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wno-m

In [7]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.0 which is incompatible.
seaborn 0.12.2 requires numpy!=1.24.0,>=1.17, but you have numpy 1.24.0 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.0 which is incompatible.


In [2]:
!ls my_model

config.json  generation_config.json  pytorch_model.bin


In [19]:
!python3 convert.py /content/my_model/

Loading model file /content/my_model/pytorch_model.bin
vocabtype: spm
Loading vocab file /content/my_model/tokenizer.model
params: n_vocab:32000 n_embd:3200 n_mult:240 n_head:32 n_layer:26
Writing vocab...
[  1/237] Writing tensor tok_embeddings.weight                  | size  32000 x   3200  | type UnquantizedDataType(name='F16')
[  2/237] Writing tensor norm.weight                            | size   3200           | type UnquantizedDataType(name='F32')
[  3/237] Writing tensor output.weight                          | size  32000 x   3200  | type UnquantizedDataType(name='F16')
[  4/237] Writing tensor layers.0.attention.wq.weight           | size   3200 x   3200  | type UnquantizedDataType(name='F16')
[  5/237] Writing tensor layers.0.attention.wk.weight           | size   3200 x   3200  | type UnquantizedDataType(name='F16')
[  6/237] Writing tensor layers.0.attention.wv.weight           | size   3200 x   3200  | type UnquantizedDataType(name='F16')
[  7/237] Writing tensor layers.

In [21]:
!./quantize /content/my_model/ggml-model-f16.bin /content/my_model/ggml-model-q4_0.bin q4_0

main: build = 916 (b5472ea)
main: quantizing '/content/my_model/ggml-model-f16.bin' to '/content/my_model/ggml-model-q4_0.bin' as Q4_0
llama.cpp: loading model from /content/my_model/ggml-model-f16.bin
llama.cpp: saving model to /content/my_model/ggml-model-q4_0.bin
[   1/ 237]                tok_embeddings.weight -     3200 x 32000, type =    f16, quantizing to q4_0 .. size =   195.31 MB ->    54.93 MB | hist: 0.036 0.016 0.025 0.039 0.057 0.077 0.096 0.111 0.117 0.111 0.096 0.077 0.057 0.039 0.025 0.021 
[   2/ 237]                          norm.weight -             3200, type =    f32, size =    0.012 MB
[   3/ 237]                        output.weight -     3200 x 32000, type =    f16, quantizing to q4_0 .. size =   195.31 MB ->    54.93 MB | hist: 0.036 0.015 0.024 0.037 0.055 0.076 0.097 0.114 0.121 0.114 0.097 0.076 0.055 0.037 0.024 0.020 
[   4/ 237]         layers.0.attention.wq.weight -     3200 x  3200, type =    f16, quantizing to q4_0 .. size =    19.53 MB ->     5.49 MB 

In [12]:
!ls /content/llama.cpp/models

ggml-vocab.bin


In [18]:
mv /content/mymodel/tokenizer.model /content/my_model/

In [23]:
!ls -lha /content/my_model/

total 8.2G
drwxr-xr-x 2 root root 4.0K Jul 27 00:55 .
drwxr-xr-x 1 root root 4.0K Jul 27 00:50 ..
-rw-r--r-- 1 root root  629 Jul 27 00:21 config.json
-rw-r--r-- 1 root root  132 Jul 27 00:21 generation_config.json
-rw-r--r-- 1 root root 6.4G Jul 27 00:54 ggml-model-f16.bin
-rw-r--r-- 1 root root 1.8G Jul 27 00:56 ggml-model-q4_0.bin
-rw-r--r-- 1 root root 1.5K Jul 27 00:21 .gitattributes
lrwxrwxrwx 1 root root  180 Jul 27 00:23 pytorch_model.bin -> ../../root/.cache/huggingface/hub/models--lemonteaa--exercise-openllama-3b-qlora-axolotl-checkpoint400-merged/blobs/f19072b44f8e6fac58490ff156683d0ff239004722811361eb214f9fdbcb902c
-rw-r--r-- 1 root root 522K Jul 27 00:50 tokenizer.model


In [25]:
from huggingface_hub import create_repo

create_repo("lemonteaa/exercise-openllama-3b-qlora-axolotl-checkpoint400-GGML", repo_type="model")

RepoUrl('https://huggingface.co/lemonteaa/exercise-openllama-3b-qlora-axolotl-checkpoint400-GGML', endpoint='https://huggingface.co', repo_type='model', repo_id='lemonteaa/exercise-openllama-3b-qlora-axolotl-checkpoint400-GGML')

In [26]:
from huggingface_hub import HfApi

api = HfApi()
api.upload_file(
    path_or_fileobj="/content/my_model/ggml-model-q4_0.bin",
    path_in_repo="openllama-3b-qlora-axolotl-ck400.ggml.q4_0.bin",
    repo_id="lemonteaa/exercise-openllama-3b-qlora-axolotl-checkpoint400-GGML",
    repo_type="model"
)

ggml-model-q4_0.bin:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

'https://huggingface.co/lemonteaa/exercise-openllama-3b-qlora-axolotl-checkpoint400-GGML/blob/main/openllama-3b-qlora-axolotl-ck400.ggml.q4_0.bin'